<a href="https://colab.research.google.com/github/Pontes-y/fiap/blob/main/fiap_ano2_fase_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# lista com os arquivos a serem analisados
arquivos = ['/content/vendas_linha_petshop_2019 - vendas_linha_petshop_2019.csv',
            '/content/vendas_linha_petshop_2020 - vendas_linha_petshop_2020.csv',
            '/content/vendas_linha_petshop_2021 - vendas_linha_petshop_2021.csv',
            '/content/vendas_linha_petshop_2022 - vendas_linha_petshop_2022.csv']

df_list = []

for arquivo in arquivos:
  df_list.append(pd.read_csv(arquivo))

df = pd.concat(df_list, ignore_index=True)

df.head()


,cod_pedido,regiao_pais,produto,valor,quantidade,valor_total_bruto,data,estado,formapagto,centro_distribuicao,responsavelpedido,valor_comissao,lucro_liquido,categoriaprod
0,1,Norte,Biscoito True Champion 300g,22,2.0,44,13/06/2019,Acre,Dinheiro,Rapid Pink,Carlos,"0,44","17,6",Alimentação
1,2,Norte,Biscoito True Champion 300g,21,2.0,42,03/01/2019,Amapá,Pix,Rapid Pink,Maria Linda,"0,42","16,8",Alimentação
2,3,Norte,Biscoito True Champion 300g,22,NaN,44,18/01/2019,Pará,Boleto Bancário,Rapid Pink,Julia,"0,88","35,2",Alimentação
3,4,Norte,Biscoito True Champion 300g,19,4.0,88,19/08/2019,Rondônia,Boleto Bancário,Rapid Pink,Yuri,"0,76","30,4",Alimentação
4,5,Norte,Biscoito True Champion 300g,22,2.0,44,18/10/2019,Tocantins,Cartão Crédito,Rapid Pink,Dolores,"0,44","17,6",Alimentação


In [2]:
# corrigindo a coluna de data e validando se todos os anos/arquivos estão no df
df['data'] = pd.to_datetime(df['data'], dayfirst=True)
anos_unicos = df['data'].dt.year.unique()

print(anos_unicos)

[2019 2020 2021 2022]


In [3]:
# corrigindo valores nulos no df
df = df.replace(np.nan, 0)

# transformando coluna de data para formato padrão
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

In [4]:
# 1.4.1 Identificando dados missing
valores_faltantes = df.isnull()

contagem_faltantes = valores_faltantes.sum()

print(contagem_faltantes)

cod_pedido             0
regiao_pais            0
produto                0
valor                  0
quantidade             0
valor_total_bruto      0
data                   0
estado                 0
formapagto             0
centro_distribuicao    0
responsavelpedido      0
valor_comissao         0
lucro_liquido          0
categoriaprod          0
dtype: int64


In [5]:
# 1.4.2 Analise dos outiliers na coluna quantidade
print("Numero de registros", df['quantidade'].count())
print("Media da quantidade", df['quantidade'].mean())
print("Variancia", df['quantidade'].var())
print("Desvio padrao", df['quantidade'].std())

# 1.96 vezes o desvio padrao
df['1.96*std'] = 1.96 * df['quantidade'].std()

# outilier true se a diferenca for maior que 1.96 vezes o desvio padrao
df['outlier'] = abs(df['quantidade'] - df['quantidade'].mean()) > df['1.96*std']

# contando a quatidade de outliers
print("Quantidade de Outliers", df['outlier'].value_counts()[True])

# Mostrando outliers
print(df[['cod_pedido', 'quantidade']].loc[df['outlier'] == True])

df.describe()

Numero de registros 200786
Media da quantidade 3.638510653133187
Variancia 144.64555808564245
Desvio padrao 12.026868174451836
Quantidade de Outliers 5972
        cod_pedido  quantidade
30              31        71.0
156            157        70.0
165            166        70.0
183            184        70.0
228            229        72.0
...            ...         ...
200511      200706        44.0
200571      200766        45.0
200579      200774        42.0
200605      200800        43.0
200630      200825        42.0

[5972 rows x 2 columns]


,cod_pedido,quantidade,1.96*std
count,200786.000000,200786.000000,200786.000000
mean,100490.520290,3.638511,23.572662
std,58046.094844,12.026868,0.000000
min,1.000000,0.000000,23.572662
25%,50197.250000,1.000000,23.572662
50%,100587.500000,1.000000,23.572662
75%,150783.750000,2.000000,23.572662
max,200980.000000,110.000000,23.572662
